In [ ]:
import numpy as np
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


class MeanAbsoluteAngularError(nn.Module):
    def forward(self, pred, target):
        diff = torch.abs(pred - target) % 360
        angular_error = torch.minimum(diff, 360 - diff)
        return angular_error.mean()


class CommonModel(nn.Module):
    def __init__(self, num_regions):
        super().__init__()
        base = resnet50(weights=ResNet50_Weights.DEFAULT)
        for param in base.parameters():
            param.requires_grad = False
        base.fc = nn.Identity()
        self.backbone = base

        self.shared_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.angle_head = nn.Linear(256, 1)
        self.coord_head = nn.Linear(256, 2)
        self.region_head = nn.Linear(256, num_regions)

    def forward(self, x):
        feat = self.backbone(x)
        feat = self.shared_head(feat)
        angle = self.angle_head(feat).squeeze(1)
        coords = self.coord_head(feat)
        region = self.region_head(feat)
        return angle, coords, region

    def evaluate(self, val_loader, device):
        self.eval()
        criterion_angle = MeanAbsoluteAngularError()
        total_maae = 0
        total_samples = 0

        with torch.no_grad():
            for imgs, angle_targets, coord_targets, region_targets in val_loader:
                imgs = imgs.to(device)
                angle_targets = angle_targets.to(device)

                angle_preds, _, _ = self(imgs)
                maae = criterion_angle(angle_preds, angle_targets)
                total_maae += maae.item() * imgs.size(0)
                total_samples += imgs.size(0)

        final_maae = total_maae / total_samples
        accuracy = 1 / (1 + final_maae)
        print(f"\nFinal Validation Accuracy (1 / (1 + MAAE)): {accuracy:.4f}")

    def train_model(self, train_loader, val_loader, epochs, lr, device='cuda', angle_weight=1.0, coord_weight=0.3, region_weight=0.3):
        self.to(device)
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion_angle = MeanAbsoluteAngularError()
        criterion_coord = nn.MSELoss()
        criterion_region = nn.CrossEntropyLoss()

        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0

            for imgs, angle_targets, coord_targets, region_targets in train_loader:
                imgs = imgs.to(device)
                angle_targets = angle_targets.to(device)
                coord_targets = coord_targets.to(device)
                region_targets = region_targets.to(device)

                optimizer.zero_grad()
                angle_preds, coord_preds, region_preds = self(imgs)

                loss_angle = criterion_angle(angle_preds, angle_targets)
                loss_coords = criterion_coord(coord_preds, coord_targets)
                loss_region = criterion_region(region_preds, region_targets)

                loss = angle_weight * loss_angle + coord_weight * loss_coords + region_weight * loss_region
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * imgs.size(0)

            avg_loss = total_loss / len(train_loader.dataset)
            print(f"Epoch {epoch}/{epochs} - Total Weighted Loss: {avg_loss:.4f}")
            self.evaluate(val_loader, device)
            
class ImageDataset(Dataset):
    def __init__(self, csv_file, images_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.images_dir, row['filename'])
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        angle = torch.tensor(row['angle'], dtype=torch.float)
        coords = torch.tensor([row['latitude'], row['longitude']], dtype=torch.float)
        region_id = torch.tensor(row['region_id'], dtype=torch.long)
        return img, angle, coords, region_id
        
train_data = ImageDataset("/kaggle/input/smai-project/labels_train_updated.csv",
                                "/kaggle/input/smai-project/images_train/images_train", 
                                transform=transform_train)

val_data = ImageDataset("/kaggle/input/smai-project/labels_val_updated.csv",
                              "/kaggle/input/smai-project/images_val/images_val", 
                              transform=transform_val)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

num_regions = 15
model = CommonModel(num_regions=num_regions)
model.train_model(train_loader, val_loader, epochs=30, lr=1e-4, device='cuda')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 231MB/s]



=== Training RESNET ===
Epoch 1/30 - Train MAAE: 0.0113

Final Validation Accuracy (1 / (1 + MAAE)): 0.0118


In [ ]:
val_preds_tensor = model.predict_angles(val_loader, device='cuda')
val_preds_np = val_preds_tensor.numpy() % 360

class TestImageDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.filenames = sorted(os.listdir(images_dir))
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img_path = os.path.join(self.images_dir, filename)
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, filename

test_dataset = TestImageDataset("/kaggle/input/smai-test/images_test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_preds = []
with torch.no_grad():
    for imgs, _ in test_loader:
        imgs = imgs.to('cuda')
        outputs = model(imgs)
        test_preds.extend(outputs.cpu().numpy())

test_preds_np = np.array(test_preds) % 360
ids = np.arange(738)
angles = np.round(np.concatenate([val_preds_np, test_preds_np])).astype(int) % 360


submission_df = pd.DataFrame({
    'id': ids,
    'angle': angles
})
submission_df.to_csv("2021102007_1.csv", index=False)